In [70]:
import pandas as pd
import numpy as np

In [71]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [72]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [73]:
from sklearn.model_selection import train_test_split

# X - матрица признаков, y - целевая переменная
X = data.drop('MEDV', axis=1)
y = data['MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=RANDOM_STATE)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((404, 13), (102, 13), (404,), (102,))

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [74]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

lr = LinearRegression()
lr.fit(X_train, y_train)


r2_lr = lr.score(X_test, y_test)

ridge = Ridge()
ridge.fit(X_train, y_train)
r2_ridge = ridge.score(X_test, y_test)

lasso = Lasso()
lasso.fit(X_train, y_train)
r2_lasso = lasso.score(X_test, y_test)

print(f"Linear regression R2 score: {r2_lr}")
print(f"Ridge regression R2 score: {r2_ridge}")
print(f"Lasso regression R2 score: {r2_lasso}")


results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

Linear regression R2 score: 0.6684825753971567
Ridge regression R2 score: 0.6659608075261693
Lasso regression R2 score: 0.6668687223368213


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [75]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import RidgeCV, LassoCV

# Создаем объект модели
ridge = Ridge()
lasso = Lasso()

# Создаем сетку параметров для перебора
param_grid = {'alpha': [10**i for i in range(-5, 6)]}

# Инициализируем объект GridSearchCV
ridge_grid_search_cv = GridSearchCV(ridge, param_grid, cv=5)
lasso_grid_search_cv = GridSearchCV(lasso, param_grid, cv=5)

# Обучаем модели на обучающей выборке
ridge_grid_search_cv.fit(X_train, y_train)
lasso_grid_search_cv.fit(X_train, y_train)

r2_ridge_grid_search = ridge_grid_search_cv.score(X_test, y_test)
r2_lasso_grid_search = lasso_grid_search_cv.score(X_test, y_test)

# Выводим наилучшие параметры и R2 на тестовой выборке
print("Ridge GridSearchCV : Best alpha =", ridge_grid_search_cv.best_params_['alpha'])
print("Lasso GridSearchCV: Best alpha =", lasso_grid_search_cv.best_params_['alpha'])


# Инициализируем объекты RidgeCV и LassoCV
ridge_cv = RidgeCV(alphas=[10**i for i in range(-5, 6)], cv=5)
lasso_cv = LassoCV(alphas=[10**i for i in range(-5, 6)], cv=5)

# Обучаем модели на обучающей выборке
ridge_cv.fit(X_train, y_train)
lasso_cv.fit(X_train, y_train)

r2_ridge_cv = ridge_cv.score(X_test, y_test)
r2_lasso_cv = lasso_cv.score(X_test, y_test)


# Выводим наилучшие параметры и R2 на тестовой выборке
print("Ridge RidgeCV: Best alpha =", ridge_cv.alpha_)
print("Lasso LassoCV: Best alpha =", lasso_cv.alpha_)


print(f"r2_ridge: {r2_ridge}")
print(f"r2_ridge_grid_search: {r2_ridge_grid_search}")
print(f"r2_ridge_cv: {r2_ridge_cv}")

print("------")

print(f"r2_lasso: {r2_lasso}")
print(f"r2_lasso_grid_search: {r2_lasso_grid_search}")
print(f"r2_lasso_cv: {r2_lasso_cv}")

results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]

Ridge GridSearchCV : Best alpha = 1e-05
Lasso GridSearchCV: Best alpha = 1e-05
Ridge RidgeCV: Best alpha = 1e-05
Lasso LassoCV: Best alpha = 1e-05
r2_ridge: 0.6659608075261693
r2_ridge_grid_search: 0.6684825680074256
r2_ridge_cv: 0.6684825680074256
------
r2_lasso: 0.6668687223368213
r2_lasso_grid_search: 0.6684829595885677
r2_lasso_cv: 0.6684829595885677


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [57]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import r2_score

# Создаем Pipeline для масштабирования данных и обучения Ridge/Lasso модели
pipeline_ridge_standard_scaler = Pipeline([('scaler', StandardScaler()), ('reg', Ridge())])
pipeline_lasso_standard_scaler = Pipeline([('scaler', StandardScaler()), ('reg', Lasso())])

# Обучаем модели
pipeline_ridge_standard_scaler.fit(X_train, y_train)
pipeline_lasso_standard_scaler.fit(X_train, y_train)

# Вычисляем R2 на тестовой выборке
y_predict_ridge_standard_scaler = pipeline_ridge_standard_scaler.predict(X_test)
y_predict_lasso_standard_scaler = pipeline_lasso_standard_scaler.predict(X_test)

r2_ridge_standart_scaler = r2_score(y_test, y_predict_ridge_standard_scaler)
r2_lasso_standart_scaler = r2_score(y_test, y_predict_lasso_standard_scaler)


# Создаем Pipeline с MinMaxScaler и Ridge/Lasso
pipeline_ridge_min_max_scaler = Pipeline([('scaler', MinMaxScaler()), ('reg', Ridge())])
pipeline_lasso_min_max_scaler = Pipeline([('scaler', MinMaxScaler()), ('reg', Lasso())])


# Обучаем модели
pipeline_ridge_min_max_scaler.fit(X_train, y_train)
pipeline_lasso_min_max_scaler.fit(X_train, y_train)

# Вычисляем R2 на тестовой выборке
y_predict_ridge_min_max_scaler = pipeline_ridge_min_max_scaler.predict(X_test)
y_predict_lasso_min_max_scaler = pipeline_lasso_min_max_scaler.predict(X_test)
r2_ridge_min_max_scaler = r2_score(y_test, y_predict_ridge_min_max_scaler)
r2_lasso_min_max_scaler = r2_score(y_test, y_predict_lasso_min_max_scaler)


print(f"r2_ridge: {r2_ridge}")
print(f"r2_ridge_grid_search: {r2_ridge_grid_search}")
print(f"r2_ridge_cv: {r2_ridge_cv}")
print(f"r2_ridge_standart_scaler: {r2_ridge_standart_scaler}")
print(f"r2_ridge_min_max_scaler: {r2_ridge_min_max_scaler}")

print("------")

print(f"r2_lasso: {r2_lasso}")
print(f"r2_lasso_grid_search: {r2_lasso_grid_search}")
print(f"r2_lasso_cv: {r2_lasso_cv}")
print(f"r2_lasso_standart_scaler: {r2_lasso_standart_scaler}")
print(f"r2_lasso_min_max_scaler: {r2_lasso_min_max_scaler}")


results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]

r2_ridge: 0.6659608075261695
r2_ridge_grid_search: 0.6684825680074258
r2_ridge_cv: 0.6684825680074258
r2_ridge_standart_scaler: 0.668190107677443
r2_ridge_min_max_scaler: 0.6762207658974593
------
r2_lasso: 0.6668687223368214
r2_lasso_grid_search: 0.668482959588568
r2_lasso_cv: 0.668482959588568
r2_lasso_standart_scaler: 0.624044752347846
r2_lasso_min_max_scaler: 0.2573921442545195


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [79]:
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Создаем объекты RidgeCV и LassoCV
ridge_cv = RidgeCV(alphas=[10**i for i in range(-5, 6)], cv=5)
lasso_cv = LassoCV(alphas=[10**i for i in range(-5, 6)], cv=5)

# Масштабируем данные
standard_scaler = StandardScaler()
X_train_standard_scaled = standard_scaler.fit_transform(X_train)
X_test_standard_scaled = standard_scaler.transform(X_test)

# Обучаем модели на масштабированных данных
ridge_cv.fit(X_train_standard_scaled, y_train)
lasso_cv.fit(X_train_standard_scaled, y_train)

# Получаем значения R2 для Ridge и Lasso на тестовой выборке
r2_ridge_standart_scaler_cv = ridge_cv.score(X_test_standard_scaled, y_test)
r2_lasso_standart_scaler_cv = lasso_cv.score(X_test_standard_scaled, y_test)

# Выводим наилучшие параметры и R2 на тестовой выборке
print("Ridge RidgeCV StandardScaler: Best alpha =", ridge_cv.alpha_)
print("Lasso LassoCV StandardScaler: Best alpha =", lasso_cv.alpha_)

# Выводим значения R2
print("r2_ridge_standart_scaler_cv: ", r2_ridge_standart_scaler_cv)
print("r2_lasso_standart_scaler_cv: ", r2_lasso_standart_scaler_cv)

print("------")

# Масштабируем данные
min_max_scaler = MinMaxScaler()
X_train_min_max_scaled = min_max_scaler.fit_transform(X_train)
X_test_min_max_scaled = min_max_scaler.transform(X_test)

# Обучаем модели на масштабированных данных
ridge_cv.fit(X_train_min_max_scaled, y_train)
lasso_cv.fit(X_train_min_max_scaled, y_train)

# Получаем значения R2 для Ridge и Lasso на тестовой выборке
r2_ridge_min_max_scaler_cv = ridge_cv.score(X_test_standard_scaled, y_test)
r2_lasso_min_max_scaler_cv = lasso_cv.score(X_test_standard_scaled, y_test)

# Выводим наилучшие параметры и R2 на тестовой выборке
print("Ridge RidgeCV MinMaxScaler: Best alpha =", ridge_cv.alpha_)
print("Lasso LassoCV MinMaxScaler: Best alpha =", lasso_cv.alpha_)

# Выводим значения R2
print("r2_ridge_min_max_scaler_cv: ", r2_ridge_min_max_scaler_cv)
print("r2_lasso_min_max_scaler_cv: ", r2_lasso_min_max_scaler_cv)


results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]

Ridge RidgeCV StandardScaler: Best alpha = 1.0
Lasso LassoCV StandardScaler: Best alpha = 1e-05
r2_ridge_standart_scaler_cv:  0.6681901076774432
r2_lasso_standart_scaler_cv:  0.6684821312777707
------
Ridge RidgeCV MinMaxScaler: Best alpha = 0.1
Lasso LassoCV MinMaxScaler: Best alpha = 1e-05
r2_ridge_min_max_scaler_cv:  -20.66034718450295
r2_lasso_min_max_scaler_cv:  -21.108987672687487


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [82]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Создаем объект PolynomialFeatures для добавления попарных произведений и квадратов признаков
poly = PolynomialFeatures(include_bias=False)

# Масштабируем данные
standard_scaler = StandardScaler()
X_train_standard_scaled = standard_scaler.fit_transform(X_train)
X_test_standard_scaled = standard_scaler.transform(X_test)

# Добавляем попарные произведения и квадраты признаков
X_train_poly = poly.fit_transform(X_train_standard_scaled)
X_test_poly = poly.transform(X_test_standard_scaled)

# Масштабируем признаки с помощью StandardScaler
X_train_scaled = standard_scaler.fit_transform(X_train_poly)
X_test_scaled = standard_scaler.transform(X_test_poly)

# Обучаем модель Ridge с параметрами по умолчанию
ridge = Ridge()
ridge.fit(X_train_scaled, y_train)
r2_ridge_standart_scaler_poly = ridge.score(X_test_scaled, y_test)
print(f'r2_ridge_standart_scaler_poly: {r2_ridge_standart_scaler_poly}')

# Обучаем модель Lasso с параметрами по умолчанию
lasso = Lasso()
lasso.fit(X_train_scaled, y_train)
r2_lasso_standart_scaler_poly = lasso.score(X_test_scaled, y_test)
print(f'r2_lasso_standart_scaler_poly: {r2_lasso_standart_scaler_poly}')

print("------")

# Масштабируем данные MinMaxScaler
min_max_scaler = MinMaxScaler()
X_train_min_max_scaled = min_max_scaler.fit_transform(X_train)
X_test_min_max_scaled = min_max_scaler.transform(X_test)

# Добавляем попарные произведения и квадраты признаков
X_train_poly = poly.fit_transform(X_train_min_max_scaled)
X_test_poly = poly.transform(X_test_min_max_scaled)

# Масштабируем признаки с помощью StandardScaler
X_train_scaled = min_max_scaler.fit_transform(X_train_poly)
X_test_scaled = min_max_scaler.transform(X_test_poly)

# Обучаем модель Ridge с параметрами по умолчанию
ridge = Ridge()
ridge.fit(X_train_scaled, y_train)
r2_ridge_min_max_scaler_poly = ridge.score(X_test_scaled, y_test)
print(f'r2_ridge_min_max_scaler_poly: {r2_ridge_min_max_scaler_poly}')

# Обучаем модель Lasso с параметрами по умолчанию
lasso = Lasso()
lasso.fit(X_train_scaled, y_train)
r2_lasso_min_max_scaler_poly = lasso.score(X_test_scaled, y_test)
print(f'r2_lasso_min_max_scaler_poly: {r2_lasso_min_max_scaler_poly}')




results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]

r2_ridge_standart_scaler_poly: 0.8172321234956241
r2_lasso_standart_scaler_poly: 0.7388607450771217
------
r2_ridge_min_max_scaler_poly: 0.8402436598340637
r2_lasso_min_max_scaler_poly: 0.2877326671559647


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

# Создаем пайплайн для масштабирования, генерации полиномиальных признаков и моделирования
pipe_standard_scaler = make_pipeline(StandardScaler(), PolynomialFeatures())

ridge_cv = RidgeCV(alphas=[10**i for i in range(-5, 6)], cv=5)
lasso_cv = LassoCV(alphas=[10**i for i in range(-5, 6)], cv=5)

# Масштабируем данные
X_train_standard_scaled = pipe_standard_scaler.fit_transform(X_train)
X_test_standard_scaled = pipe_standard_scaler.transform(X_test)

# Обучаем модели на масштабированных данных
ridge_cv.fit(X_train_standard_scaled, y_train)
lasso_cv.fit(X_train_standard_scaled, y_train)

# Получаем значения R2 для Ridge и Lasso на тестовой выборке
r2_ridge_standart_scaler_poly_cv = ridge_cv.score(X_test_standard_scaled, y_test)
r2_lasso_standart_scaler_poly_cv = lasso_cv.score(X_test_standard_scaled, y_test)

# Выводим наилучшие параметры и R2 на тестовой выборке
print("Ridge RidgeCV StandardScaler: Best alpha =", ridge_cv.alpha_)
print("Lasso LassoCV StandardScaler: Best alpha =", lasso_cv.alpha_)

# Выводим значения R2
print(f'r2_ridge_standart_scaler_poly_cv: {r2_ridge_standart_scaler_poly_cv}')
print(f'r2_lasso_standart_scaler_poly_cv: {r2_lasso_standart_scaler_poly_cv}')
print("------")


pipe_min_max_scaler = make_pipeline(MinMaxScaler(), PolynomialFeatures())
X_train_min_max_scaled = pipe_min_max_scaler.fit_transform(X_train)
X_test_min_max_scaled = pipe_min_max_scaler.transform(X_test)
ridge_cv = RidgeCV(alphas=[10**i for i in range(-5, 6)], cv=5)
lasso_cv = LassoCV(alphas=[10**i for i in range(-5, 6)], cv=5)
ridge_cv.fit(X_train_min_max_scaled, y_train)
lasso_cv.fit(X_test_min_max_scaled, y_train)
r2_ridge_min_max_scaler_poly_cv = ridge_cv.score(X_test_min_max_scaled, y_test)
r2_lasso_min_max_scaler_poly_cv = lasso_cv.score(X_test_min_max_scaled, y_test)
print("Ridge RidgeCV MinMaxScaler: Best alpha =", ridge_cv.alpha_)
print("Lasso LassoCV MinMaxScaler: Best alpha =", lasso_cv.alpha_)
print(f'r2_ridge_min_max_scaler_poly_cv: {r2_ridge_min_max_scaler_poly_cv}')
print(f'r2_lasso_min_max_scaler_poly_cv: {r2_lasso_min_max_scaler_poly_cv}')


results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]

8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [85]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV

# Создаем Pipeline
pipe = Pipeline([
    ('scaler', StandardScaler()), 
    ('poly', PolynomialFeatures()), 
    ('reg', Ridge())
])

# Создаем сетку параметров, которые будем перебирать в GridSearchCV
param_grid = {
    'poly__degree': [1, 2, 3],  # Степень полинома
    'scaler': [StandardScaler(), MinMaxScaler()],  # Метод масштабирования
    'reg': [Ridge(), Lasso()],  # Тип регуляризации
    'reg__alpha': [0.1, 1, 10]  # Коэффициент регуляризации
}

# Создаем GridSearchCV с заданными параметрами
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='r2')

# Обучаем модель и выводим лучшие параметры и результат R2
grid.fit(X_train, y_train)

print('Параметры лучшей модели:\n', grid.best_params_)
r2_best_model = grid.best_score_
print('r2_best_model', r2_best_model)
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]

Параметры лучшей модели:
 {'poly__degree': 3, 'reg': Ridge(alpha=0.1), 'reg__alpha': 0.1, 'scaler': MinMaxScaler()}
r2_best_model 0.86624814908432


In [86]:
results_regression

,model,task,R2
0,LR,task2,0.668483
1,Ridge,task2,0.665961
2,Lasso,task2,0.666869
3,Ridge_GridSearchCV,task3,0.668483
4,RidgeCV,task3,0.668483
5,Lasso_GridSearchCV,task3,0.668483
6,LassoCV,task3,0.668483
11,Ridge_StandardScaler_CV,task5,0.668190
12,Ridge_MinMaxScaler_CV,task5,-20.660347
13,Lasso_StandardScaler_CV,task5,0.668482


http://archive.ics.uci.edu/ml/datasets/Adult

In [6]:
data = pd.read_csv('adult.csv')
data

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,39,Private,215419,Bachelors,13,Divorced,Prof-specialty,Not-in-family,White,Female,0,0,36,United-States,<=50K
48838,64,?,321403,HS-grad,9,Widowed,?,Other-relative,Black,Male,0,0,40,United-States,<=50K
48839,38,Private,374983,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,50,United-States,<=50K
48840,44,Private,83891,Bachelors,13,Divorced,Adm-clerical,Own-child,Asian-Pac-Islander,Male,5455,0,40,United-States,<=50K


9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [25]:
data = pd.read_csv('adult.csv')

# Замена целевой переменной на числовые значения
data['class'] = data['class'].map({'<=50K': 1, '>50K': 0})

# Выделение признаков и целевой переменной
X = data.drop('class', axis=1)
y = data['class']

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [22]:
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

data = pd.read_csv('adult.csv')
data['class'] = data['class'].map({'<=50K': 1, '>50K': 0})
X = data.drop('class', axis=1)
y = data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

# предсказание самого частого класса
y_pred = np.zeros_like(y_test) + y_train.value_counts().idxmax()

# y_test - истинные значения целевой переменной на тестовой выборке
# y_pred - предсказанные значения

acc_most_frequent = accuracy_score(y_test, y_pred)
f1_most_frequent = f1_score(y_test, y_pred)
print(f"acc_most_frequent: {acc_most_frequent}")
print(f"f1_most_frequent: {f1_most_frequent}")

results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]

acc_most_frequent: 0.7581382652016652
f1_most_frequent: 0.8624330409129726


11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [47]:
from sklearn.impute import SimpleImputer

data = pd.read_csv('adult.csv')
data['class'] = data['class'].map({'<=50K': 1, '>50K': 0})
X = data.drop('class', axis=1)
y = data['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

# X = X.replace("?", np.NaN)
# Создаем объект класса SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')

# Применяем imputer к датафрейму X
X_imputed = imputer.fit_transform(X)
X_imputed

array([[39, 'State-gov', 77516, ..., 0, 40, 'United-States'],
       [50, 'Self-emp-not-inc', 83311, ..., 0, 13, 'United-States'],
       [38, 'Private', 215646, ..., 0, 40, 'United-States'],
       ...,
       [38, 'Private', 374983, ..., 0, 50, 'United-States'],
       [44, 'Private', 83891, ..., 0, 40, 'United-States'],
       [35, 'Self-emp-inc', 182148, ..., 0, 60, 'United-States']],
      dtype=object)

12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [41]:
data = pd.read_csv('adult.csv')
data['class'] = data['class'].map({'<=50K': 1, '>50K': 0})
X = data.drop('class', axis=1)
y = data['class']

num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()


print(f"numerical_columns: {num_cols}")
print(f"categorical_columns: {cat_cols}")

numerical_columns: ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_columns: ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']


13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [63]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler , OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC


data = pd.read_csv('adult.csv')
data['class'] = data['class'].map({'<=50K': 1, '>50K': 0})

X = data.drop('class', axis=1)
y = data['class']

# Определяем числовые и категориальные признаки
num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()

# Пайплайн для обработки числовых признаков
numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

# Пайплайн для обработки категориальных признаков
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Объединяем числовой и категориальный пайплайны
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])


# Define classifiers
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
knn = KNeighborsClassifier()
svc = LinearSVC()

# Define pipeline with column transformer and classifiers
pipe_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', lr)
])

pipe_knn = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', knn)
])

pipe_svc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', svc)
])

# Calculate cross-validation scores using accuracy and F1 metrics
acc_LR = cross_val_score(pipe_lr, X, y, cv=5, scoring='accuracy').mean()
f1_LR = cross_val_score(pipe_lr, X, y, cv=5, scoring='f1').mean()

acc_KNN = cross_val_score(pipe_knn, X, y, cv=5, scoring='accuracy').mean()
f1_KNN = cross_val_score(pipe_knn, X, y, cv=5, scoring='f1').mean()

acc_SVM = cross_val_score(pipe_svc, X, y, cv=5, scoring='accuracy').mean()
f1_SVM = cross_val_score(pipe_svc, X, y, cv=5, scoring='f1').mean()

print(f"acc_LR: {acc_LR}")
print(f"acc_KNN: {acc_KNN}")
print(f"acc_SVM: {acc_SVM}")

print(f"f1_LR: {f1_LR}")
print(f"f1_KNN: {f1_KNN}")
print(f"f1_SVM: {f1_SVM}")
                                           
                                           
results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]

acc_LR: 0.8508252089273688
acc_KNN: 0.8247409969643567
acc_SVM: 0.8529135478362626
f1_LR: 0.9047857586982012
f1_KNN: 0.8869692965663359
f1_SVM: 0.9063223083526415


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [65]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.impute import SimpleImputer


data = pd.read_csv('adult.csv')
data['class'] = data['class'].map({'<=50K': 1, '>50K': 0})
data = data.replace('?', np.nan)

X = data.drop('class', axis=1)
y = data['class']

# Определяем числовые и категориальные признаки
num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()


# Пайплайн для обработки числовых признаков
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', MinMaxScaler())
])

# Пайплайн для обработки категориальных признаков
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Объединяем числовой и категориальный пайплайны
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])


# Define classifiers
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
knn = KNeighborsClassifier()
svc = LinearSVC()

# Define pipeline with column transformer and classifiers
pipe_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', lr)
])

pipe_knn = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', knn)
])

pipe_svc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', svc)
])

# Calculate cross-validation scores using accuracy and F1 metrics
acc_LR = cross_val_score(pipe_lr, X, y, cv=5, scoring='accuracy').mean()
f1_LR = cross_val_score(pipe_lr, X, y, cv=5, scoring='f1').mean()

acc_KNN = cross_val_score(pipe_knn, X, y, cv=5, scoring='accuracy').mean()
f1_KNN = cross_val_score(pipe_knn, X, y, cv=5, scoring='f1').mean()

acc_SVM = cross_val_score(pipe_svc, X, y, cv=5, scoring='accuracy').mean()
f1_SVM = cross_val_score(pipe_svc, X, y, cv=5, scoring='f1').mean()

print(f"acc_LR: {acc_LR}")
print(f"acc_KNN: {acc_KNN}")
print(f"acc_SVM: {acc_SVM}")

print(f"f1_LR: {f1_LR}")
print(f"f1_KNN: {f1_KNN}")
print(f"f1_SVM: {f1_SVM}")
                                           
                                           
results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]

acc_LR: 0.8507023483249301
acc_KNN: 0.8252118868046804
acc_SVM: 0.8512551277675652
f1_LR: 0.9047791034209263
f1_KNN: 0.8873196076146141
f1_SVM: 0.9054218744827377


15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [61]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC


data = pd.read_csv('adult.csv')
data = data.replace('?', np.nan)
data = data.dropna()
# data.mask(data.eq('?')).dropna()
data['class'] = data['class'].map({'<=50K': 1, '>50K': 0})
X = data.drop('class', axis=1)
y = data['class']


# Определяем числовые и категориальные признаки
num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()

# Пайплайн для обработки числовых признаков
numeric_transformer = Pipeline(steps=[
    ('scaler', MinMaxScaler())
])

# Пайплайн для обработки категориальных признаков
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Объединяем числовой и категориальный пайплайны
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])


# Define classifiers
lr = LogisticRegression(solver='lbfgs', max_iter=1000)
knn = KNeighborsClassifier()
svc = LinearSVC()

# Define pipeline with column transformer and classifiers
pipe_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', lr)
])

pipe_knn = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', knn)
])

pipe_svc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', svc)
])

# Calculate cross-validation scores using accuracy and F1 metrics
acc_LR_del_missings = cross_val_score(pipe_lr, X, y, cv=5, scoring='accuracy').mean()
f1_LR_del_missings = cross_val_score(pipe_lr, X, y, cv=5, scoring='f1').mean()

acc_KNN_del_missings = cross_val_score(pipe_knn, X, y, cv=5, scoring='accuracy').mean()
f1_KNN_del_missings = cross_val_score(pipe_knn, X, y, cv=5, scoring='f1').mean()

acc_SVM_del_missings = cross_val_score(pipe_svc, X, y, cv=5, scoring='accuracy').mean()
f1_SVM_del_missings = cross_val_score(pipe_svc, X, y, cv=5, scoring='f1').mean()

print(f"acc_LR_del_missings: {acc_LR_del_missings}")
print(f"acc_KNN_del_missings: {acc_KNN_del_missings}")
print(f"acc_SVM_del_missings: {acc_SVM_del_missings}")

print(f"f1_LR_del_missings: {f1_LR_del_missings}")
print(f"f1_KNN_del_missings: {f1_KNN_del_missings}")
print(f"f1_SVM_del_missings: {f1_SVM_del_missings}")


results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

acc_LR_del_missings: 0.8467339991770471
acc_KNN_del_missings: 0.8205298095497255
acc_SVM_del_missings: 0.8485030154158197
f1_LR_del_missings: 0.9010789016007372
f1_KNN_del_missings: 0.8829256825024464
f1_SVM_del_missings: 0.9024033119235104


 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [68]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.impute import SimpleImputer


data = pd.read_csv('adult.csv')
data['class'] = data['class'].map({'<=50K': 1, '>50K': 0})
data = data.replace('?', np.nan)

X = data.drop('class', axis=1)
y = data['class']

# Определяем числовые и категориальные признаки
num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = X.select_dtypes(include='object').columns.tolist()


# Пайплайн для обработки числовых признаков
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', MinMaxScaler())
])

# Пайплайн для обработки категориальных признаков
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Объединяем числовой и категориальный пайплайны
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_cols),
        ('cat', categorical_transformer, cat_cols)
    ])

# Define classifiers
rfc = RandomForestClassifier()
gbc = GradientBoostingClassifier()

# Define pipeline with column transformer and classifiers
pipe_rfc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', rfc)
])

pipe_gbc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', gbc)
])


# Calculate cross-validation scores using accuracy and F1 metrics
acc_RF = cross_val_score(pipe_rfc, X, y, cv=5, scoring='accuracy').mean()
f1_RF = cross_val_score(pipe_rfc, X, y, cv=5, scoring='f1').mean()

acc_GB = cross_val_score(pipe_gbc, X, y, cv=5, scoring='accuracy').mean()
f1_GB = cross_val_score(pipe_gbc, X, y, cv=5, scoring='f1').mean()


print(f"acc_RF: {acc_RF}")
print(f"acc_GB: {acc_GB}")

print(f"f1_RF: {f1_RF}")
print(f"f1_GB: {f1_GB}")

results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

acc_RF: 0.8514394113355113
acc_GB: 0.8665903354382216
f1_RF: 0.9040587148361044
f1_GB: 0.9155015819476015


In [ ]:
acc_RF: 0.8514394113355113
acc_GB: 0.8665903354382216
f1_RF: 0.9040587148361044
f1_GB: 0.9155015819476015

17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV

data = pd.read_csv('adult.csv')
data['class'] = data['class'].map({'<=50K': 1, '>50K': 0})
data = data.replace('?', np.nan)

X = data.drop('class', axis=1)
y = data['class']

# Определение шагов для обработки числовых и категориальных признаков
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('scaler', MinMaxScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Объединение шагов в пайплайн
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Определение классификаторов
classifiers = [
    LogisticRegression(),
    KNeighborsClassifier(),
    LinearSVC(),
    RandomForestClassifier(),
    GradientBoostingClassifier()
]

# Определение пайплайна
pipe = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', classifiers)])

# Определение сетки параметров для подбора
param_grid = [
    {
        'preprocessor__num__imputer__strategy': ['mean', 'median'],
        'classifier': classifiers
    },
    {
        'preprocessor__num__imputer__strategy': ['mean', 'median'],
        'preprocessor__cat__onehot__handle_unknown': ['error', 'ignore'],
        'classifier': classifiers
    }
]

# Подбор наилучшей модели
grid_search = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid_search.fit(X, y)

# Вывод результатов
acc_best = grid_search.best_score_

best_params = grid_search.best_params_
print('Параметры лучшей модели:\n', best_params)

results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

In [69]:
results_classification

,model,task,f1,accuracy
0,Most Frequent class,task10,0.862433,0.758138
1,LogisticRegression,task13,0.904786,0.850825
2,KNeighborsClassifier,task13,0.886969,0.824741
3,LinearSVC,task13,0.906322,0.852914
7,LogisticRegression_delete_missings,task15,0.901079,0.846734
8,KNeighborsClassifier_delete_missings,task15,0.882926,0.82053
9,LinearSVC_delete_missings,task15,0.902403,0.848503
4,LogisticRegression_impute,task14,0.904779,0.850702
5,KNeighborsClassifier_impute,task14,0.88732,0.825212
6,LinearSVC_impute,task14,0.905422,0.851255
